# DIP opdracht 2: Letterfrequenties
Lucas Siekkötter, 1770991
&
Wijnand van Dijk, 1762733 

## Imports

In [1]:
import numpy
import string
from numpy.matrixlib.defmatrix import matrix

## Code

### info.py
Maakt een lijst met alle leestekens, die lijst kunnen we later gebruiken om makkelijk alle leesteken te vervangen. Ook wordt er een lijst aangemaakt van ons 'alfabet' die bestaat uit het normale alfabet plus '_' en '%'.

In [2]:
leestekens = [".", ",", ":", ";", "\'", "\"", "-", "_", "!", "?", "(", ")", "\\"]

abc = list(string.ascii_lowercase)
abc.extend(["_", "%"])

### mapper.py
De functie 'mapper_get_lines' lijst de hele tekst uit. 

'mapper_replace_punct' zorgt ervoor dat alle leestekens en spaties vervangen worden, '_' inplaats van een spatie en '%' inplaats van een leesteken.

In [3]:
# Mappers
def mapper_get_lines():
    """
    
    """
    res = []
    with open("data/validation.txt", "r", encoding="utf8") as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line != "":
                res.append(line)
    
    return res

def mapper_replace_punct(text, sign):
    """
    Replace all punctuation with % and replace spaces with _
    @param  sign: leestekens
    @return     : text waar leestekens vervangen zijn
    """
    res = []
    for line in text:
        for i in sign:
            line = line.replace(i, "%")
        
        line = line.replace(" ", "_")
        line = line.lower()
        res.append(line)

    return res

### filter.py
'filter_training' geeft een lijst van alle letter combinaties die in de zin voorkomen, dus niet de combinaties die een leesteken of spatie bevatten (of wat daar voor in de plaats staat).

'filter_validation' doet eigenlijk hetzelfde alleen geeft hij het in een andere form terug.

In [4]:
# Filter
def filter_training(txt):
    """
    Get all 2 letter combinations in text
    @param  txt: tekst waat combinaties uit gehaald worden
    @return    : lijst met combinaties in tekst
    """
    combinations = []
    for line in txt:
        for letter in range(0, len(line)):  
            try:
                res = line[letter] + line[letter + 1]
                # leesteken + spatie is niet relevant dus die combinaties kunnen eruit. "\n" zitten er ook tussen en die kunnen ook weg
                # en check of beide letters wel in het (aangepaste) alfabet zitten
                if (res[0] in abc and res[-1] in abc) and (res != "%_" and res != "_%" and "\n" not in res):
                    combinations.append([res])
            except:
                continue
    
    return combinations


def filter_validation(txt):
    """
    Get all 2 letter combinations in text.
    Different funtion was made to return in correct form
    @param  txt: tekst waat combinaties uit gehaald worden
    @return    : lijst met combinaties in tekst
    """
    combinations = []
    for line in txt:
        temp = []
        for letter in range(0, len(line)):  
            try:
                res = line[letter] + line[letter + 1]
                if (res[0] in abc and res[-1] in abc) and (res != "%_" and res != "_%" and "\n" not in res):
                    temp.append(res)
            except:
                continue
        combinations.append(temp)
    
    return combinations

### reducer.py
'reducer' ??

'reducer_chance_matrix' ??

'reducer_final' 
De matrices zijn gevuld met kansen. Per combinatie die in een zin voorkomt
pakt het algoritme de bijbehorende kans en telt die bij de totale kans dat een
zin nederlands of engels is op. De taal met het hoogste getal aan het einde 
wordt als waarheid genomen.

In [5]:
# Reduce
def reducer(c):
    """
    """
    matrix = numpy.zeros((28, 28))
    for combination in c:
        left, right = combination[0], combination[-1]
        try:
            matrix[abc.index(left)][abc.index(right)] += 1 # find right place in matrix and add 1
        except:
            # double check: if one of the letters isnt in the alphabet skip this one and continue
            continue 
    
    return matrix


def reducer_chance_matrix(m):
    """
    Convert a frequence matrix to a chance matrix
    """
    total = numpy.sum(m) # total letter combinations analysed
    
    with numpy.nditer(m, op_flags=['readwrite']) as it:
        for x in it:
            if x != 0:
                x[...] = round(x / total * 100, 4) # chance x to its chance that it occurs
            
    return m
    



def reducer_final(eng_m, nl_m, c):
    """
    
    @param eng_m: english trained matrix numpy array
    @param nl_m: dutch trained matrix numpy array
    @param c: list van combinaties die voorkomen per zin
    """
    nl_count = 0    # houdt bij hoeveel nederlandse zinnen er zijn gevonden
    eng_count = 0   # houdt bij hoveel engelse zinnen er zijn gevonden
    for line in c:  # loopt door de tekst heen per line
        eng = 0   
        nl = 0
        for combination in line:
            left, right = abc.index(combination[0]), abc.index(combination[-1]) # vind juiste index voor matrices
            oem = eng_m[left][right]    # On English Matrix. De kans dat de combinatie voorkomt in het engels
            odm = nl_m[left][right]     # On Dutch Matrix. De kans dat de combinatie voorkomt in het engels
            eng += oem  # tel de kans op bij het totaal (voor sepcifieke zin)
            nl += odm
        
        # er is nu een complete zin gecheckt -> kijk welk getal groter is en tel één bij de telling op
        # Een gelijkspel zal zeer waarschijnlijk niet voorkomen omdat de kansen op 3 decimalen zijn afgerond
        if eng > nl:
            eng_count += 1
        else:
            nl_count += 1
        
        #reset waardes voor zin en ga door naar de volgende zin
        eng, nl = 0, 0  
    
    return nl_count, eng_count

### train_matrix.py
asdass

In [6]:
def train_matrix(txt):
    """
    """
    matrix = numpy.zeros((28, 28))
    for line in txt:
        m = reducer(line)
        matrix = numpy.add(matrix, m)
    
    chance_m = reducer_chance_matrix(matrix)

    numpy.save("eng_trained", chance_m)

def train():

    lines_list = mapper_get_lines()
    updated_text = mapper_replace_punct(lines_list, leestekens)
    combi_list = filter_training(updated_text)

    train_matrix(combi_list)

### main.py
asdasda

In [7]:
def main():
    """
    Main function
    """

    lines_list = mapper_get_lines()
    updated_text = mapper_replace_punct(lines_list, leestekens)
    combi_list = filter_validation(updated_text)

    nl, eng = numpy.load('trained_matrix/nl_trained.npy'), numpy.load('trained_matrix/eng_trained.npy')
    result = reducer_final(eng, nl, combi_list)
    print(f'In deze tekst zijn {result[0]} Nederlandse zinnen gevonden, en {result[1]} Engelse zinnen.')

main()

In deze tekst zijn 74 Nederlandse zinnen gevonden, en 117 Engelse zinnen.
